### Objective:
Data exploration to construct a sensible hypothesis. \
Course module assignment.\
First kaggle notebook.

### Statement: Education industry directly impacts all sectors. [Plot at line20]

Ho = Growth of education sector has no impact on other sectors.\
Ha = Growth of Education sector has a positive correlation with other sectors.

### Index:
Prerequisites\
Working out the data\
Basic Visalization

### Prerequisites

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#Importing the dataset
data = pd.read_csv('/kaggle/input/all-indian-companies-registration-data-1900-2019/registered_companies.csv')

In [ ]:
df = data

### Working the data

In [ ]:
df.info()

-There are a lot of categoricals to work with \
-Many columns seem broadly empty \

In [ ]:
print(f"Total Values : {len(df)}\n")
for x in df.columns:
    print(f'{len(df)-df[x].count()} values missing in {x}')

In [ ]:
print(len(df))
df.head(3)

In [ ]:
df = df.dropna()
print(len(df))
df.head(3)

In [ ]:
#Interesting data, will keep it narrow and efficient for now
dropCols = ["LATEST_YEAR_FINANCIAL_STATEMENT",
            "EMAIL_ADDR", "COMPANY_NAME",
            "LATEST_YEAR_ANNUAL_RETURN",
            "CORPORATE_IDENTIFICATION_NUMBER",
            "REGISTERED_OFFICE_ADDRESS"]

df = df.drop(dropCols, axis=1)

In [ ]:
df["DATE_OF_REGISTRATION"] = df["DATE_OF_REGISTRATION"].apply(pd.to_datetime)
#df["INDUSTRIAL_CLASS"] = df["INDUSTRIAL_CLASS"].astype(int)

In [ ]:
df['REG_YEAR'] = df['DATE_OF_REGISTRATION'].dt.year
df['REG_MONTH'] = df['DATE_OF_REGISTRATION'].dt.month

In [ ]:
#columns along the number of unique items in them along a list of it
for x in df.columns:
    print(f'{x} : {len(df[x].unique())}\n{df[x].unique()[:20]}\n')

In [ ]:
#Naming conveniences

df["COMPANY_CLASS"] = df["COMPANY_CLASS"].apply(lambda x:"Solo" if x == 'Private(One Person Company)' else x)
print(df["COMPANY_CLASS"].unique())

df["REGISTRAR"] = df["REGISTRAR_OF_COMPANIES"].apply(lambda x:x.split("ROC\xa0")[-1])
print(df["REGISTRAR"].unique())

df["PRINCIPAL_BUSINESS"] = df["PRINCIPAL_BUSINESS_ACTIVITY_AS_PER_CIN"].apply(lambda x:x.split(" ")[0])
print(df["PRINCIPAL_BUSINESS"].unique())

#Creating bins for REGISTRATION_YEAR

df["REG_YEAR_5BIN"] = df["REG_YEAR"].apply(lambda x:(round(x/5))*5)
print(df["REG_YEAR_5BIN"].unique())

df["REG_YEAR_10BIN"] = df["REG_YEAR"].apply(lambda x:(round(x/10))*10)
print(df["REG_YEAR_10BIN"].unique())
      
df["REG_YEAR_20BIN"] = df["REG_YEAR"].apply(lambda x:(round(x/20))*20)
print(df["REG_YEAR_20BIN"].unique())

### Visualization

In [ ]:
#Working with a smaller randomly picked sample space for efficiency and overall population testing.
df2 = df
df = df.sample(n=10000)

In [ ]:
#For readability since I use dark mode
sns.set_theme(context='notebook',
              style='darkgrid',
              palette='magma',
              font='sans-serif',
              font_scale=0.6,
              color_codes=True,
              rc=None)

In [ ]:
f, ax = plt.subplots(2)

#Counting all the number fo companies by REG_YEAR

sns.countplot(x="REG_YEAR_5BIN",
              data=df, ax = ax[0])


#Year of registration by COMPANY_CLASS

sns.stripplot(x="REG_YEAR",
              y="COMPANY_CLASS",
              data=df, jitter=0.5,
              ax = ax[1])

In [ ]:
f, ax = plt.subplots(1, len(df["COMPANY_CLASS"].unique()))
f.tight_layout()
y=0
print(df["COMPANY_CLASS"].unique())
for x in df["COMPANY_CLASS"].unique():
    sns.histplot(x="REG_YEAR",
                data=df[df["COMPANY_CLASS"]==x],
                ax=ax[y])
    y+=1

India is seeing a increasing number of new companies being registered which a vast proportion of them being in the 2000s+ \
Solo COMPANY_CLASS catches traction post 2010+ from the first lower plot and can see appearing from 2014 in this sample space on the second rightmost plot.\
The majority of companies are classified as Private (density of the first lower plot & count in second plot).



In [ ]:
sns.countplot(x="COMPANY_CLASS",
              data=df[df["REG_YEAR"] >= 1982]) #Change year

In [ ]:
#Industry of companies

sns.stripplot(x="REG_YEAR",
              y="PRINCIPAL_BUSINESS",
              hue="COMPANY_CLASS",
              data=df, jitter=0.3)

The density of private companies is too high to make any reasonable inference.

In [ ]:
#Companies in respect to COMPANY_CLASS over time.
y=0
f, ax = plt.subplots(len(df["COMPANY_CLASS"].unique()),1)
f.subplots_adjust(top=1, bottom=-0.9, left=-0.9, hspace=0.2)
print(df["COMPANY_CLASS"].unique())
for x in df["COMPANY_CLASS"].unique():
    sns.stripplot(y="REG_YEAR",
                x="PRINCIPAL_BUSINESS",
                data=df[df["COMPANY_CLASS"]==x],
                hue="REG_YEAR_10BIN",
                palette="magma_r",
                jitter=0.4,
                ax=ax[y])
    y+=1

-Wholesale, Manufacturing and Real Estate have the highest number private companies.\
-Public companies incline towards manufacturing followed by Real Estate and Financial sectors.\
-Interesting how private companies have a big concentration in Real Estate even in the sparcity.\
-Companies under Education and electricity industry started late in India and are relevant magnifications to look at.

In [ ]:
#State wise looking at the industries through time

y=0; f, ax = plt.subplots(len(df["REGISTERED_STATE"].unique()),1)
f.subplots_adjust(top=10, bottom=-0.9, left=-0.5, hspace=0.2)
for x in df["REGISTERED_STATE"].unique():
    sns.histplot(y='REG_YEAR',
                  x='PRINCIPAL_BUSINESS',
                  #jitter=0.3,
                  hue="REGISTERED_STATE",
                  palette="magma_r",
                  data= df[df["REGISTERED_STATE"]==x],
                  ax=ax[y])
    y+=1

### Education industry directly impacts all sectors?

In [ ]:
len(df)

In [ ]:
df[df["PRINCIPAL_BUSINESS"] == "Education"].head(3)

In [ ]:
f, axes = plt.subplots(1,5)
f.subplots_adjust(top=0.5, bottom=-0.9, left=-3, hspace=0.2)
ax1 = sns.countplot(x="COMPANY_CLASS", data=df2[df2["PRINCIPAL_BUSINESS"] == "Education"], ax=axes[0])
ax2 = sns.countplot(x="REGISTERED_STATE", data=df2[df2["PRINCIPAL_BUSINESS"] == "Education"], ax=axes[1])
ax3 = sns.countplot(x="REG_YEAR_10BIN", data=df2[df2["PRINCIPAL_BUSINESS"] == "Education"], ax=axes[2])
ax4 = sns.countplot(x="COMPANY_CATEGORY", data=df2[df2["PRINCIPAL_BUSINESS"] == "Education"], ax=axes[3])
ax5 = sns.countplot(x="COMPANY_SUB_CATEGORY", data=df2[df2["PRINCIPAL_BUSINESS"] == "Education"], ax=axes[4])
ax2.set_xticklabels(ax2.get_xticklabels(), rotation=40, ha="right")
ax4.set_xticklabels(ax4.get_xticklabels(), rotation=40, ha="right")
ax5.set_xticklabels(ax5.get_xticklabels(), rotation=40, ha="right")

Using the df2 full set DataFrame the result and distribution changes quite a bit

In [ ]:
print(len(df), len(df2))

In [ ]:
sns.displot(x="REG_YEAR", y="PRINCIPAL_BUSINESS", data=df[df["PRINCIPAL_BUSINESS"] == "Education"])

In [ ]:
sns.displot(x="REG_YEAR", y="PRINCIPAL_BUSINESS", data=df[df["PRINCIPAL_BUSINESS"] != "Education"])

In [ ]:
df["AUTHORIZED_CAP"].describe()

In [ ]:
df.to_csv('/kaggle/working/df.csv')

In [ ]:
df2.head()